<a href="https://colab.research.google.com/github/gretagh93/ChainsLarp/blob/master/Thinc_Ivy_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thinc - Ivy - Simulation

For this POC I would like to simulate the Ivy's implementation of the function min() in Thinc

In [ ]:
!pip install "thinc>=8.0.0" "ml_datasets>=0.2.0" "tqdm>=4.41"
!pip install git+https://github.com/unifyai/ivy.git

In [2]:
import thinc.util
# If you want to try out the PyTorch integration, you'll need to install it.
if not thinc.util.has_torch:
    !pip install "torch"

import thinc.util
# If you want to run this notebook on GPU, you'll need to install cupy.
if not thinc.util.has_cupy:
    !pip install "cupy-cuda101"

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 10.4 MB/s eta 0:00:00


In [24]:
import ivy
import torch
import tensorflow as tf

I will use several of Ivy's backends to demonstrate the different functions of the Backend Handler, in specific:

*   unset_backend()
*   current_backend()
*   set_backend()

In [19]:
# No backend
ivy.unset_backend()
backend = ivy.current_backend()
print(backend)

x = ivy.array([5, 2, 3])
z = ivy.min(x)
print(z)

<module 'ivy.functional.backends.numpy' from '/usr/local/lib/python3.9/dist-packages/ivy/functional/backends/numpy/__init__.py'>
ivy.array(2)


In [20]:
# PyTorch backend
ivy.set_backend('torch')
backend = ivy.current_backend()
print(backend)

x = ivy.array([5, 2, 3])
z = ivy.min(x)
print(z)

<module 'ivy.functional.backends.torch' from '/usr/local/lib/python3.9/dist-packages/ivy/functional/backends/torch/__init__.py'>
ivy.array(2)


In [22]:
# TensorFlow backend
ivy.set_backend('tensorflow')
backend = ivy.current_backend()
print(backend)

x = ivy.array([5, 2, 3])
z = ivy.min(x)
print(z)

<module 'ivy.functional.backends.tensorflow' from '/usr/local/lib/python3.9/dist-packages/ivy/functional/backends/tensorflow/__init__.py'>
ivy.array(2)


## Simulation



To do the simulation we need to explore the code of min() function in:


*   [Ivy functional API](https://github.com/unifyai/ivy/blob/master/ivy/functional/ivy/statistical.py)
*   [TensorFlow](https://github.com/unifyai/ivy/blob/master/ivy/functional/backends/tensorflow/statistical.py) and [PyTorch](https://github.com/unifyai/ivy/blob/master/ivy/functional/backends/torch/statistical.py) specific backend of Backend Functional API


TensorFlow specific backend

In [25]:
def min(
    x: ivy.Union[tf.Tensor, tf.Variable],
    /,
    *,
    axis: ivy.Optional[ivy.Union[int, ivy.Sequence[int]]] = None,
    keepdims: bool = False,
    out: ivy.Optional[ivy.Union[tf.Tensor, tf.Variable]] = None,
) -> ivy.Union[tf.Tensor, tf.Variable]:
    axis = tuple(axis) if isinstance(axis, list) else axis
    return tf.math.reduce_min(x, axis=axis, keepdims=keepdims)

AttributeError: ignored

PyTorch specific backend

In [ ]:
def min(
    x: torch.Tensor,
    /,
    *,
    axis: ivy.Optional[ivy.Union[int, ivy.Sequence[int]]] = None,
    keepdims: bool = False,
    out: ivy.Optional[torch.Tensor] = None,
) -> torch.Tensor:
    if axis == ():
        if ivy.exists(out):
            return ivy.inplace_update(out, x)
        else:
            return x
    if not keepdims and not axis and axis != 0:
        return torch.amin(input=x, out=out)
    return torch.amin(input=x, dim=axis, keepdim=keepdims, out=out)


min.support_native_out = True

Ivy functional API function

In [ ]:
def min(
    x: ivy.Union[ivy.Array, ivy.NativeArray],
    /,
    *,
    axis: ivy.Optional[ivy.Union[int, ivy.Sequence[int]]] = None,
    keepdims: bool = False,
    out: ivy.Optional[ivy.Array] = None,
) -> ivy.Array:
    """Calculates the minimum value of the input array ``x``.

    .. note::
       When the number of elements over which to compute the minimum value is zero, the
       minimum value is implementation-defined. Specification-compliant libraries may
       choose to raise an error, return a sentinel value (e.g., if ``x`` is a floating-point
       input array, return ``NaN``), or return the maximum possible value for the input array ``x``
       data type (e.g., if ``x`` is a floating-point array, return ``+infinity``).

    **Special Cases**

    For floating-point operands,

    -   If ``x_i`` is ``NaN``, the minimum value is ``NaN`` (i.e., ``NaN`` values propagate).

    Parameters
    ----------
    x
        Input array. Should have a real-valued data type.
    axis
        axis or axes along which minimum values must be computed. By default, the
        minimum value must be computed over the entire array. If a tuple of integers,
        minimum values must be computed over multiple axes. Default: ``None``.

    keepdims
        optional boolean, if ``True``, the reduced axes (dimensions) must be included in the
        result as singleton dimensions, and, accordingly, the result must be compatible
        with the input array (see :ref:`broadcasting`). Otherwise, if ``False``, the reduced axes
        (dimensions) must not be included in the result. Default: ``False``.
    out
        optional output array, for writing the result to.

    Returns
    -------
    ret
        if the minimum value was computed over the entire array, a zero-dimensional
        array containing the minimum value; otherwise, a non-zero-dimensional array
        containing the minimum values. The returned array must have the same data type
        as ``x``.


    This function conforms to the `Array API Standard
    <https://data-apis.org/array-api/latest/>`_. This docstring is an extension of the
    `docstring <https://data-apis.org/array-api/latest/API_specification/generated/signatures.statistical_functions.min.html>`_  # noqa
    in the standard.

    Both the description and the type hints above assumes an array input for simplicity,
    but this function is *nestable*, and therefore also accepts :class:`ivy.Container`
    instances in place of any of the arguments.

    Examples
    --------
    With :class:`ivy.Array` input:

    >>> x = ivy.array([1, 2, 3])
    >>> z = ivy.min(x)
    >>> print(z)
    ivy.array(1)

    >>> x = ivy.array([0, 1, 2])
    >>> z = ivy.array([0, 0, 0])
    >>> y = ivy.min(x, out=z)
    >>> print(z)
    ivy.array(0)

    >>> x = ivy.array([[0, 1, 2], [4, 6, 10]])
    >>> y = ivy.min(x, axis=0, keepdims=True)
    >>> print(y)
    ivy.array([[0, 1, 2]])

    >>> x = ivy.native_array([[0, 1, 2], [4, 6, 10]])
    >>> y = ivy.min(x)
    >>> print(y)
    ivy.array(0)

    With :class:`ivy.Container` input:

    >>> x = ivy.Container(a=ivy.array([1, 2, 3]), b=ivy.array([2, 3, 4]))
    >>> z = ivy.min(x)
    >>> print(z)
    {
        a: ivy.array(1),
        b: ivy.array(2)
    }
    """
    return ivy.current_backend(x).min(x, axis=axis, keepdims=keepdims, out=out)
